In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import KFold
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
warnings.filterwarnings(action='ignore')

In [3]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')

In [26]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
  error[person_idx-train_user_id_min,err-1] +=1

problem = np.zeros(15000)
problem[train_problem['user_id'].unique()-train_user_id_min] = 1

train_x = error
train_y = problem
print(train_x.shape)
print(train_y.shape)

100%|██████████| 16554663/16554663 [00:44<00:00, 371651.17it/s]

(15000, 42)
(15000,)


In [27]:
rf_model = RandomForestClassifier(random_state=15)
rf_model.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=15, verbose=0,
                       warm_start=False)

In [28]:
from sklearn.metrics import accuracy_score
y_pred = rf_model.predict(train_x)
accuracy = accuracy_score(train_y,y_pred)
accuracy

0.9996666666666667

In [14]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
  test_x[person_idx-test_user_id_min,err-1]+=1
test_x = test_x.reshape(test_x.shape[0],-1)
test_x.shape

100%|██████████| 16532648/16532648 [00:44<00:00, 370157.65it/s]


(14999, 42)

In [23]:
pred_y_list =[]
pred_y = rf_model.predict(test_x)
pred_y_list.append(pred_y.reshape(-1,1))
pred_ensemble = np.mean(pred_y_list,axis=0)
pred_ensemble

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]])

In [24]:
sample_submission['problem'] = pred_ensemble
sample_submission.head()

,user_id,problem
0,30000,1.0
1,30001,0.0
2,30002,0.0
3,30003,1.0
4,30004,1.0


In [25]:
sample_submission.to_csv('/content/drive/MyDrive/데이콘/submission/rf_submission.csv')